In [1]:
import pandas as pd
from pandas import Series, DataFrame
import folium
from folium.plugins import MarkerCluster
map1 = folium.Map(location=[33.389548, 126.239267],zoom_start=15)
mc = MarkerCluster().add_to(map1)

In [2]:
def mydef(array, i):
    data = pd.read_csv(array, encoding='cp949')
    
    if i == 37:
        data['장소명'] = '관람길'
    
    Ldata = data[['위도','경도','장소명']]
    
    mc = MarkerCluster().add_to(map1)
    
    for idx, row in Ldata.iterrows():
        loc = [row[0],row[1]]
        folium.Marker(loc, popup=row[2]).add_to(mc)

In [3]:
for i in range(1,52):
    st = "C:/Users/dwshi/Downloads/제주/csv파일/52곳/"+str(i).zfill(3)+"-2.csv"
    mydef(st, i)

In [4]:
map1

In [6]:
Ldata = pd.DataFrame(columns=['아이디', '위도', '경도', '장소명'])

def mydef2(array):
    global Ldata
    data = pd.read_csv(array, encoding='cp949')
    data = data[['아이디','위도','경도','장소명']]
    Ldata = pd.concat([Ldata, data])

for i in range(1,52):
    if i==37:
        continue
    st = "C:/Users/dwshi/Downloads/제주/csv파일/52곳/"+str(i).zfill(3)+"-2.csv"
    mydef2(st)
    
Ldata

,아이디,위도,경도,장소명
0,1,33.390696,126.238435,주차장[시작]
1,1,33.390693,126.238430,주차장[시작]
2,1,33.390321,126.237903,매표소
3,1,33.390315,126.237905,매표소
4,1,33.390077,126.237525,화장실/장애인화장실
...,...,...,...,...
30,51,33.391924,126.677207,NaN
31,51,33.392221,126.677083,배수로 및 오르막 길
32,51,33.392225,126.677087,배수로 및 오르막 길
33,51,33.393206,126.676350,현위치 ⑨


In [7]:
LPdata = Ldata.dropna()
LPdata.isnull().values.any()

False

In [8]:
import geojson

def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["경도"],
                                                    X["위도"])),
                            properties=dict(name=X["아이디"],
                                            description=X["장소명"])))
    df.apply(insert_features, axis=1)
    with open('map1.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

data2geojson(LPdata)

In [9]:
map2 = folium.Map(location=[33.389548, 126.239267],zoom_start=15)

In [10]:
dd = LPdata['아이디'].value_counts().sort_index()
md = pd.DataFrame({'아이디':dd.index, 'count':dd.values})
Ldata = pd.merge(Ldata, md)
Ldata

,아이디,위도,경도,장소명,count
0,1,33.390696,126.238435,주차장[시작],201
1,1,33.390693,126.238430,주차장[시작],201
2,1,33.390321,126.237903,매표소,201
3,1,33.390315,126.237905,매표소,201
4,1,33.390077,126.237525,화장실/장애인화장실,201
...,...,...,...,...,...
3107,51,33.391924,126.677207,NaN,30
3108,51,33.392221,126.677083,배수로 및 오르막 길,30
3109,51,33.392225,126.677087,배수로 및 오르막 길,30
3110,51,33.393206,126.676350,현위치 ⑨,30


In [11]:
folium.Choropleth(
    geo_data="map1.geojson", data=LPdata, columns=('아이디','count'),
    key_on='feature.properties.name', fill_color='BuPu', legend_name='위치').add_to(map2)
map2

KeyError: 'count'